# 03 | 图文联合（CLIP）

本 Notebook 讲解如何使用 **CLIP** 进行图文检索与对齐训练。
我们会：

1. 准备图文数据
2. 使用 CLIP 计算相似度
3. 做一个小型的对齐训练
4. 评估检索效果


## 0. 环境准备

**做什么**：安装 CLIP 相关依赖。
**为什么**：CLIP 模型来自 Hugging Face。
**结果**：可以直接加载预训练模型。

In [ ]:
!pip install torch torchvision transformers pillow


## 1. 数据准备

**做什么**：准备一组图像 + 文本描述。
**为什么**：CLIP 需要成对的图文数据。
**结果**：得到一个小型数据集，后续可直接跑通。


In [ ]:
import os
import numpy as np
from PIL import Image

os.makedirs('data/clip', exist_ok=True)

samples = [
    ('red_lesion.png', [180, 40, 40], 'a red lesion in a medical image'),
    ('green_tissue.png', [40, 180, 40], 'a green tissue sample'),
    ('blue_marker.png', [40, 40, 180], 'a blue marker in the scan'),
]

for filename, color, _ in samples:
    img = np.ones((224, 224, 3), dtype=np.uint8) * np.array(color, dtype=np.uint8)
    Image.fromarray(img).save(os.path.join('data/clip', filename))

texts = [s[2] for s in samples]
image_paths = [os.path.join('data/clip', s[0]) for s in samples]
print('Toy data ready:', image_paths)


## 2. 加载 CLIP 并计算相似度

**做什么**：把图像和文本编码为向量，然后计算相似度。
**为什么**：CLIP 本质上是“图文对齐”的向量空间。
**结果**：我们可以做图文检索。


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_id = 'openai/clip-vit-base-patch32'
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)

images = [Image.open(p).convert('RGB') for p in image_paths]
inputs = processor(text=texts, images=images, return_tensors='pt', padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

# 计算相似度矩阵
similarity = image_embeds @ text_embeds.T
print('Similarity matrix:
', similarity.cpu().numpy())


## 3. 简化版对齐训练

**做什么**：使用对比损失让正确的图文配对更相似。
**为什么**：这是 CLIP 的核心训练思想。
**结果**：你可以在自己的医学图文数据上微调。


In [ ]:
import torch.nn.functional as F

model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

for step in range(3):
    inputs = processor(text=texts, images=images, return_tensors='pt', padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

    logits = image_embeds @ text_embeds.T
    labels = torch.arange(len(images), device=device)
    loss_i2t = F.cross_entropy(logits, labels)
    loss_t2i = F.cross_entropy(logits.T, labels)
    loss = (loss_i2t + loss_t2i) / 2

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Step {step} Loss: {loss.item():.4f}')


## 4. 简单检索评估

**做什么**：检索每个文本最匹配的图像。
**为什么**：验证图文对齐是否有效。
**结果**：输出最匹配的图片文件名。


In [ ]:
model.eval()
inputs = processor(text=texts, images=images, return_tensors='pt', padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

similarity = image_embeds @ text_embeds.T
best_matches = similarity.argmax(dim=0).cpu().numpy()

for i, text in enumerate(texts):
    matched_path = image_paths[best_matches[i]]
    print(f'Text: {text} -> Image: {matched_path}')
